In [1]:
from tensorflow.keras import layers
import tensorflow as tf

class Patches(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super().__init__()
        self.patch_size = patch_size

    def get_config(self):
        config = super().get_config().copy()
        config.update({"input_shape": input_shape,"patch_size": patch_size,"num_patches": num_patches,"projection_dim": projection_dim,"num_heads": num_heads,"transformer_units": transformer_units,"transformer_layers": transformer_layers,"mlp_head_units": mlp_head_units,})
        return config

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(images=images,sizes=[1, self.patch_size, self.patch_size, 1],strides=[1, self.patch_size, self.patch_size, 1],rates=[1, 1, 1, 1],padding="VALID",)
        return tf.reshape(patches, [batch_size, -1, patches.shape[-1]])

class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim, **kwargs):
        super().__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(input_dim=num_patches, output_dim=projection_dim)
    def get_config(self):
        config = super().get_config().copy()
        config.update({"input_shape": input_shape,"patch_size": patch_size,"num_patches": num_patches,"projection_dim": projection_dim,"num_heads": num_heads,"transformer_units": transformer_units,"transformer_layers": transformer_layers,"mlp_head_units": mlp_head_units,})
        return config

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded
    
def Annotate(image, annotations, true=None):
    annotated_image = image.copy()
    img = ImageDraw.Draw(annotated_image)
    for bounding_box in annotations:
        img.rectangle(bounding_box, outline="white", width=3)
    if true != None:
        img.rectangle(true, outline="red", width=3)
    annotated_image.show()
    
def intersection_over_union(box_predicted, box_truth):
    top_x_intersect = max(box_predicted[0], box_truth[0])
    top_y_intersect = max(box_predicted[1], box_truth[1])
    bottom_x_intersect = min(box_predicted[2], box_truth[2])
    bottom_y_intersect = min(box_predicted[3], box_truth[3])
    intersection_area = max(0, bottom_x_intersect - top_x_intersect + 1) * max(0, bottom_y_intersect - top_y_intersect + 1)
    box_predicted_area = (box_predicted[2] - box_predicted[0] + 1) * (box_predicted[3] - box_predicted[1] + 1)
    box_truth_area = (box_truth[2] - box_truth[0] + 1) * (box_truth[3] - box_truth[1] + 1)
    return intersection_area / float(box_predicted_area + box_truth_area - intersection_area)

model = tf.keras.models.load_model("PCBAoI_on_ViT.h5", custom_objects={'Patches': Patches, 'PatchEncoder': PatchEncoder}, compile=False)

C:\Users\M10901K01\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\M10901K01\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\M10901K01\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [10]:
from PIL import Image, ImageDraw
import xml.etree.ElementTree as ET
import numpy as np
   
image_size = 224
image = tf.keras.utils.load_img('./test_data/JPEGImages/20170314-SPI-AOI-1.jpeg')
(w, h) = image.size[:2]
image = image.resize((image_size, image_size))
root = ET.parse('./test_data/Annotations/20170314-SPI-AOI-1.xml').getroot()
for neighbor in root.iter('bndbox'):
    xmin, ymin, xmax, ymax = int(neighbor.find('xmin').text), int(neighbor.find('ymin').text), int(neighbor.find('xmax').text), int(neighbor.find('ymax').text)
    annot_true = np.array([xmin, ymin, xmax, ymax])
    
annot_pred = model.predict(np.expand_dims(image, axis=0))[0]
pred = [int(annot_pred[0] * image_size), int(annot_pred[1] * image_size), int(annot_pred[2] * image_size), int(annot_pred[3] * image_size)]
true = [int(annot_true[0] / w * image_size), int(annot_true[1] / h * image_size), int(annot_true[2] / w * image_size), int(annot_true[3] / h * image_size)]
Annotate(image, [pred], true)
pred, true

1/1 [==============================] - 0s 28ms/step


([63, 144, 19, 145], [166, 193, 173, 204])